In [1]:
import numpy as np

# Choose the "true" parameters.
m_true = -0.9594
b_true = 4.294
f_true = 0.534

# Generate some synthetic data from the model.
N = 50
x = np.sort(10*np.random.rand(N))
yerr = 0.1+0.5*np.random.rand(N)
y = m_true*x+b_true
y += np.abs(f_true*y) * np.random.randn(N)
y += yerr * np.random.randn(N)

In [6]:
def lnlike(theta, x, y, yerr):
    m, b, lnf = theta
    model = m * x + b
    inv_sigma2 = 1.0/(yerr**2 + model**2*np.exp(2*lnf))
    return -0.5*(np.sum((y-model)**2*inv_sigma2 - np.log(inv_sigma2)))
import scipy.optimize as op
nll = lambda *args: -lnlike(*args)
result = op.minimize(nll, [m_true, b_true, np.log(f_true)], args=(x, y, yerr))
m_ml, b_ml, lnf_ml = result["x"]

In [7]:
def lnprior(theta):
    m, b, lnf = theta
    if -5.0 < m < 0.5 and 0.0 < b < 10.0 and -10.0 < lnf < 1.0:
        return 0.0
    return -np.inf

In [8]:
def lnprob(theta, x, y, yerr):
    lp = lnprior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + lnlike(theta, x, y, yerr)

In [9]:
ndim, nwalkers = 3, 100
pos = [result["x"] + 1e-4*np.random.randn(ndim) for i in range(nwalkers)]

In [10]:
import emcee
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=(x, y, yerr))

In [11]:
sampler.run_mcmc(pos, 500)

(array([[-0.91121659,  4.34332727, -0.63886663],
        [-1.18847191,  5.36506209, -0.65365254],
        [-1.00599829,  4.55648345, -0.81040801],
        [-0.83555467,  3.78770908, -0.45142045],
        [-1.05626418,  4.8160135 , -0.79706643],
        [-0.96515645,  4.28004971, -0.68592747],
        [-0.95159806,  4.24677352, -0.66671982],
        [-0.98802896,  4.60531244, -0.53155957],
        [-0.98708512,  4.50971849, -0.5937035 ],
        [-1.17788989,  5.39360615, -0.85049407],
        [-1.12391745,  5.05382159, -0.73886914],
        [-0.93530809,  4.2737082 , -0.37864802],
        [-1.00285885,  4.6985787 , -0.65550778],
        [-0.95680778,  4.36913541, -0.59806308],
        [-1.02408551,  4.61883248, -0.49188311],
        [-0.87904178,  4.18783015, -0.41650309],
        [-1.07386347,  4.97498167, -0.78642707],
        [-1.08192559,  4.7813503 , -0.72612783],
        [-1.07622482,  4.85737589, -0.69536665],
        [-0.91252015,  4.10617256, -0.59154323],
        [-1.00738497

In [12]:
samples = sampler.chain[:, 50:, :].reshape((-1, ndim))


In [13]:
samples[:, 2] = np.exp(samples[:, 2])
m_mcmc, b_mcmc, f_mcmc = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),
                             zip(*np.percentile(samples, [16, 50, 84],
                                                axis=0)))

In [14]:
print(m_mcmc, b_mcmc, f_mcmc)

(-1.0158209139865568, 0.08201190961103677, 0.0850266929869985) (4.586936559017882, 0.3764171467323969, 0.35888323768045893) (0.5328582269835721, 0.0872074738829709, 0.06757545430300516)
